In [60]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/shashadhar/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/shashadhar/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [ ]:
lines=[]
with open("Brown_train.txt") as file:
    while (line := file.readline().rstrip()):
        lines.append(line)
        #print(line)
print(lines[:2])    

test_lines=lines[:1]

In [61]:
sentences=[]
for line in lines:
    sentence=[]
    token_tags=line.split()
    for token_tag in token_tags:
        tok_list = token_tag.split("/")
        if(len(tok_list)==2):
            t = tuple(tok_list)
            sentence.append(t) 
    sentences.append(sentence)    
    #print(sentence) 
print(sentences[:2])        
    
 

[[('At', 'ADP'), ('that', 'DET'), ('time', 'NOUN'), ('highway', 'NOUN'), ('engineers', 'NOUN'), ('traveled', 'VERB'), ('rough', 'ADJ'), ('and', 'CONJ'), ('dirty', 'ADJ'), ('roads', 'NOUN'), ('to', 'PRT'), ('accomplish', 'VERB'), ('their', 'DET'), ('duties', 'NOUN'), ('.', '.')], [('Using', 'VERB'), ('privately-owned', 'ADJ'), ('vehicles', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('personal', 'ADJ'), ('hardship', 'NOUN'), ('for', 'ADP'), ('such', 'ADJ'), ('employees', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('the', 'DET'), ('matter', 'NOUN'), ('of', 'ADP'), ('providing', 'VERB'), ('state', 'NOUN'), ('transportation', 'NOUN'), ('was', 'VERB'), ('felt', 'VERB'), ('perfectly', 'ADV'), ('justifiable', 'ADJ'), ('.', '.')]]


In [62]:
#print each word with its respective tag for first two sentences
for sent in sentences[:2]:
    for tup in sent:
        print(tup)

('At', 'ADP')
('that', 'DET')
('time', 'NOUN')
('highway', 'NOUN')
('engineers', 'NOUN')
('traveled', 'VERB')
('rough', 'ADJ')
('and', 'CONJ')
('dirty', 'ADJ')
('roads', 'NOUN')
('to', 'PRT')
('accomplish', 'VERB')
('their', 'DET')
('duties', 'NOUN')
('.', '.')
('Using', 'VERB')
('privately-owned', 'ADJ')
('vehicles', 'NOUN')
('was', 'VERB')
('a', 'DET')
('personal', 'ADJ')
('hardship', 'NOUN')
('for', 'ADP')
('such', 'ADJ')
('employees', 'NOUN')
(',', '.')
('and', 'CONJ')
('the', 'DET')
('matter', 'NOUN')
('of', 'ADP')
('providing', 'VERB')
('state', 'NOUN')
('transportation', 'NOUN')
('was', 'VERB')
('felt', 'VERB')
('perfectly', 'ADV')
('justifiable', 'ADJ')
('.', '.')


In [63]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(sentences,train_size=0.80,test_size=0.20,random_state = 101)


In [64]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
#print(train_tagged_words)
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

434935
108117


In [65]:
# check some of the tagged words.
train_tagged_words[:5]
print(len(train_tagged_words))


434935


In [ ]:
test = train_tagged_words[7390:7394]
print(train_tagged_words[7393])

tags = {tag for word,tag in test}
print(len(tags))
print(tags)
#print(test)

In [ ]:
test = train_tagged_words[:8013]
tags = {tag for word,tag in test}
print(len(tags))
print(tags)

In [66]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'ADJ', 'PRON', 'PRT', 'ADP', 'CONJ', '.', 'ADV', 'NOUN', 'X', 'VERB', 'NUM', 'DET'}


In [67]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [68]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [69]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
    print(tags_matrix)

[[5.6893304e-02 4.7268369e-03 2.0654222e-02 9.2002057e-02 3.9116286e-02
  1.1197123e-01 1.2056859e-02 6.3240284e-01 4.7953418e-04 1.8290803e-02
  5.3776330e-03 6.0284296e-03]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000

[[5.6893304e-02 4.7268369e-03 2.0654222e-02 9.2002057e-02 3.9116286e-02
  1.1197123e-01 1.2056859e-02 6.3240284e-01 4.7953418e-04 1.8290803e-02
  5.3776330e-03 6.0284296e-03]
 [8.8772364e-03 6.2554618e-03 2.4607884e-02 5.2159514e-02 1.1912975e-02
  1.1705993e-01 5.4827284e-02 9.1532124e-03 4.5996043e-05 6.9610411e-01
  8.2792877e-04 1.8168438e-02]
 [1.9643709e-02 8.5096229e-03 1.4156195e-02 1.0227454e-01 1.5428662e-02
  9.6866548e-02 4.5649752e-02 3.0539207e-02 0.0000000e+00 5.8080167e-01
  4.2150468e-03 8.1915066e-02]
 [7.7001996e-02 8.3271585e-02 1.5806973e-02 2.0689655e-02 2.2228556e-03
  7.9604825e-03 1.6357936e-02 2.4141730e-01 3.7997530e-04 4.0448371e-02
  2.8783130e-02 4.6565974e-01]
 [1.0348196e-01 8.1990093e-02 2.7632406e-02 7.0825480e-02 2.7911519e-04
  2.2398995e-02 9.2317358e-02 2.1498848e-01 6.2800921e-04 2.1875654e-01
  2.1770986e-02 1.4493057e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000

[[5.68933040e-02 4.72683692e-03 2.06542220e-02 9.20020565e-02
  3.91162857e-02 1.11971229e-01 1.20568592e-02 6.32402837e-01
  4.79534181e-04 1.82908028e-02 5.37763303e-03 6.02842961e-03]
 [8.87723640e-03 6.25546183e-03 2.46078838e-02 5.21595143e-02
  1.19129755e-02 1.17059931e-01 5.48272841e-02 9.15321242e-03
  4.59960429e-05 6.96104109e-01 8.27928772e-04 1.81684382e-02]
 [1.96437091e-02 8.50962289e-03 1.41561953e-02 1.02274537e-01
  1.54286623e-02 9.68665481e-02 4.56497520e-02 3.05392072e-02
  0.00000000e+00 5.80801666e-01 4.21504676e-03 8.19150656e-02]
 [7.70019963e-02 8.32715854e-02 1.58069730e-02 2.06896551e-02
  2.22285558e-03 7.96048250e-03 1.63579360e-02 2.41417304e-01
  3.79975303e-04 4.04483713e-02 2.87831295e-02 4.65659738e-01]
 [1.03481963e-01 8.19900930e-02 2.76324060e-02 7.08254799e-02
  2.79115193e-04 2.23989952e-02 9.23173577e-02 2.14988485e-01
  6.28009206e-04 2.18756542e-01 2.17709858e-02 1.44930571e-01]
 [3.55329961e-02 1.29118919e-01 4.09231260e-02 1.01348408e-01
  8

In [70]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,ADJ,PRON,PRT,ADP,CONJ,.,ADV,NOUN,X,VERB,NUM,DET
ADJ,0.056893,0.004727,0.020654,0.092002,0.039116,0.111971,0.012057,0.632403,0.000480,0.018291,0.005378,0.006028
PRON,0.008877,0.006255,0.024608,0.052160,0.011913,0.117060,0.054827,0.009153,0.000046,0.696104,0.000828,0.018168
PRT,0.019644,0.008510,0.014156,0.102275,0.015429,0.096867,0.045650,0.030539,0.000000,0.580802,0.004215,0.081915
ADP,0.077002,0.083272,0.015807,0.020690,0.002223,0.007960,0.016358,0.241417,0.000380,0.040448,0.028783,0.465660
CONJ,0.103482,0.081990,0.027632,0.070825,0.000279,0.022399,0.092317,0.214988,0.000628,0.218757,0.021771,0.144931
.,0.035533,0.129119,0.040923,0.101348,0.087010,0.148970,0.081794,0.122857,0.001396,0.091510,0.018665,0.140859
ADV,0.125768,0.054004,0.030143,0.140591,0.020246,0.203091,0.091649,0.030324,0.000090,0.225868,0.012744,0.065483
NOUN,0.012170,0.019424,0.018152,0.243172,0.060312,0.288262,0.028780,0.133104,0.000352,0.171115,0.009190,0.015968
X,0.006173,0.002058,0.016461,0.039095,0.024691,0.298354,0.014403,0.067901,0.444444,0.078189,0.000000,0.008230
VERB,0.055460,0.066860,0.071045,0.164836,0.015127,0.088674,0.102953,0.092136,0.000209,0.182813,0.007285,0.152601


In [71]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [72]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [73]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  283.6536338329315
Viterbi Algorithm Accuracy:  92.10526315789474


In [74]:
#Code to test all the test sentences
#(takes alot of time to run s0 we wont run it here)
# tagging the test sentences()
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words
 
start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

KeyboardInterrupt: 